In [ ]:
import os
import sys
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.prepare_data as pd
import utils.convert_xyz_uvd as xyzuvd
import utils.visualize as visual

In [ ]:
dataset_dir = '/media/aaron/SHAREDDATA/First_Person_Action_Benchmark'

In [ ]:
train_pairs, test_pairs = pd.get_fpha_data_list('color', dataset_dir)
file_name = [i for i,j in train_pairs]
xyz_gt = [j for i,j in train_pairs]
xyz_gt = np.reshape(xyz_gt, (-1, 21, 3))
uvd_gt = xyzuvd.xyz2uvd_color(xyz_gt)

In [ ]:
i=15920
cur_frame = file_name[i]
visual.show_img_and_skel_color(cur_frame, uvd_gt[i])

rsz_dim = 256

color = Image.open(file_name[i])
color = np.asarray(color, dtype='uint32')

xyz_hand_gt = xyz_gt[i].copy()
uvd_hand_gt = uvd_gt[i].copy()

pad = 50
x_max = int(np.amax(uvd_hand_gt[:,0])) + pad
x_min = np.maximum(int(np.amin(uvd_hand_gt[:,0])) - pad, 0)
y_max = int(np.amax(uvd_hand_gt[:,1])) + pad
y_min = np.maximum(int(np.amin(uvd_hand_gt[:,1])) - pad, 0)

crop_hand = color[y_min:y_max, x_min:x_max, :]

if crop_hand.size == 0:
    print('no hand')

uvd_hand_gt[:, 0] = uvd_hand_gt[:, 0] - x_min
uvd_hand_gt[:, 1] = uvd_hand_gt[:, 1] - y_min

crop_hand_rsz = resize(crop_hand, (rsz_dim, rsz_dim), order=3, preserve_range=True).astype('uint32')

Rx = rsz_dim/crop_hand.shape[1]
Ry = rsz_dim/crop_hand.shape[0]

uvd_hand_gt_rsz = uvd_hand_gt.copy()
uvd_hand_gt_rsz[:, 0] = uvd_hand_gt_rsz[:, 0]*Rx
uvd_hand_gt_rsz[:, 1] = uvd_hand_gt_rsz[:, 1]*Ry

scoremap = pd.create_multiple_gaussian_map(uvd_hand_gt_rsz, (rsz_dim, rsz_dim))

xyz_hand_gt_norm, root_bone_length = pd.norm_keypoint(np.expand_dims(xyz_hand_gt, axis=0))
xyz_hand_gt_canon, inv_rot_mat_i = pd.canonical_transform(xyz_hand_gt_norm)
rot_mat_i = np.linalg.inv(inv_rot_mat_i)
xyz_hand_gt_canon = np.squeeze(xyz_hand_gt_canon)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(scoremap[:, :, 0])
ax.set_title('scoremap_gt')